# Monte Carlo Method : Action - Value estimation

**공부할 내용**
1. state, action, reward 가 무엇인가요?
2. Return이 무엇인가요? Gt = Rt+1 + gamma*Gt+1, 할인된 미래가치 총합
3. State-Value Function 이란? v(s) = E[Gt | St=s], 상태s에서의 미래가치(평균값)
4. Action-Value Function 이란? Q(s,a) = E[Gt | St=s, At=a], 상태s에서 액션a를 했을경우 미래가치(평균값)  
5. Policy 란? 어떤 행동을 할 확률

**사용할 환경 : 자주 만날 Frozen Lake**

![좋은거](https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcSKrspq5EvLOC2Rv8dfsw1OVqD1WjN5YsmgAHVCVn3x8zLhM5Y8&usqp=CAU)


# 필요 라이브러리 불러오기.

In [1]:
!pip install gym

In [2]:
import numpy as np
import gym

In [3]:
gym.envs.registration.register(
    id="FrozenLake-v3", entry_point='gym.envs.toy_text:FrozenLakeEnv',
    kwargs={'map_name': '4x4', 'is_slippery': False}
)

# gym.envs.registration.register(
#     id="FrozenLake-v8", entry_point='gym.envs.toy_text:FrozenLakeEnv',
#     kwargs={'map_name': '8x8', 'is_slippery': True}
# )

In [4]:
env = gym.make('FrozenLake-v3')
# env = gym.make('FrozenLake-v8')
print('observation space:', env.observation_space)
print('action space:', env.action_space)

observation space: Discrete(16)
action space: Discrete(4)


# 일단, 랜덤하게 진행한 한 번의 Episode를 기록해볼까?!

* Action 도 기록한다!!

In [5]:
states = []
actions = []
rewards = []

s0 = env.reset()
env.render()
while True :
    a0 = env.action_space.sample()
    s1, r1, done, _ = env.step(a0) # 움직여!

    states.append(s0)
    actions.append(a0)
    rewards.append(r1)
    env.render()

    if done == True :
        env.close() # 환경닫기
        break

    s0 = s1


SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG


In [6]:
states

[0, 4, 8, 4]

In [7]:
rewards

[0.0, 0.0, 0.0, 0.0]


# Return 만들기 연습!

In [9]:
# Return : Gt = Rt+1 + gamma * Gt+1

Returns = [  ]
rewards_sim = [1,2,7,-1,2,-4,5,2,2,-100]
gamma = 0.9

Gt = 0
##### 1.0 참고하여 제작!
for r in rewards_sim[::-1]:
  Gt = r + gamma * Gt
  Returns.insert(0, Gt)

print(Returns)

[-27.576075680000006, -31.751195200000005, -37.501328, -49.44592, -53.8288, -62.032000000000004, -64.48, -77.2, -88.0, -100.0]


# State,Action 별로 가치를 관찰해보자.

In [11]:
sim_returns = Returns[:len(states)]
sim_returns

[-27.576075680000006, -31.751195200000005, -37.501328, -49.44592]

In [12]:
len(states),len(actions), len(sim_returns)

(4, 4, 4)

In [13]:
print(states)
print(actions)
print(sim_returns)

[0, 4, 8, 4]
[1, 1, 3, 2]
[-27.576075680000006, -31.751195200000005, -37.501328, -49.44592]


# Action-Value (Q)를 일단 제작해보자.

V[3][2] 이라고 한다면, 3번 state에서 2번 action을 했을 때의 가치가 나와야 한다!

In [14]:
print(env.observation_space)

Discrete(16)


In [15]:
Q = np.zeros(shape=(16,4))
Q

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

# Incrementally 계산!

In [17]:
sa_n = np.zeros(shape=(16,4)) # (state , action)에 대응하는 총 갯수
Q = np.zeros(shape=(16,4))
#states, actions, sim_returns 이용하는 것은 맞다.
for t, state in enumerate(states):
    at = actions[t]
    st = state
    sa_n[st][at] = sa_n[st][at] + 1
    n_sa = sa_n[st][at]

    # Action_State_Function Q(s, a) = E [ Qt | St=s, At=a ]
    Q[st][at] = Q[st][at] + (1/n_sa)*(sim_returns[t] - Q[st][at])

Q

array([[  0.        , -27.57607568,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ],
       [  0.        , -31.7511952 , -49.44592   ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        , -37.501328  ],
       [  0.        ,   0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ],
       [  0.        ,   0

## Total 연습!

1. 총 10000번의 episode를 시행한다!
    1. 하나의 episode가 진행 중일 땐,
        * states = [] # state 기록!
        * actions = [] # action 기록
        * rewards = [] # reward 기록!
    2. 하나의 episode가 종료되면
        * returns = [] # $G_t$ 제작, 기록!
        * (사실 이 단계에서 $V(s)$를 업데이트 해야 좋음 - episode마다 업데이트가 되니까!)
2. episode가 끝나면
    * sa_n : state-action별 방문횟수 업데이트 하는 넘파이 어레이 제작
    * Q : state-action별 가치값이 저장될 0으로 가득찬 2차원 array제작
    * states, sa_n, returns 이용하여 V업데이트!

$$Q(S_t, A_t) = Q(S_t, A_t) + {1\over{N(S_t,A_t)}}(G_t - Q(S_t, A_t))$$

In [18]:
####### Your Code Here ##########

states = []
actions = []
returns = []

gamma = 0.9

for i in range(10000) :
    # 환경을 초기화 하며 초기 state 를 s0에 선언하시오.
    s0 = env.reset() 
    rewards = []
    temp_returns = []

    while True :
        # 랜덤한 액션을 하나 선택하여 a0에 선언하시오.
        a0 = env.action_space.sample()

        # a0를 이용하여 환경과 상호작용 하면서, s1, r1, done, _ 를 선언하시오.
        s1, r1, done, _ = env.step(a0)

        # states에 s0을 담고 rewards에 r1을 담으시오.
        # a0 도 actions에 담으시오.
        states.append(s0)
        rewards.append(r1)
        actions.append(a0)

        if done == True :
            env.close() # 환경닫기
            break
        # 다음 단계의 s0에 s1의 값을 옮겨 선언하시오.
        s0 = s1

    ### 이 episode에 대해서 Return을 계산하여 temp_returns에 담고
    ### returns = returns + temp_returns
    ### temp_returns.append()아님 주의! insert!
    Gt = 0
    for r in rewards[::-1] : 
        # Gt 계산! : Gt = Rt+1 + gamma * Gt+1
        Gt = r + gamma * Gt
        temp_returns.insert(0, Gt)

    returns = returns + temp_returns

In [19]:
len(states), len(returns)

(76682, 76682)

In [24]:
sa_n = np.zeros(shape=(16,4)) #  (state, action) 별 Count
Q = np.zeros(shape=(16,4))
#state, returns 이용
for t, state in enumerate(states):
    at = actions[t]    # t 시점의 액션을 선언
    st = state         # t 시점의 state를 선언해둠
    sa_n[st][at] = sa_n[st][at] + 1     # 등장횟수를 업데이트
    n_sa = sa_n[st][at]        # t시점까지, (s,a)가 등장한 쌍을 여기에 선언해옴. (윗 줄 이용!)
    Q[st][at] = Q[st][at] + (1/n_sa) * (returns[t] - Q[st][at])  # 업데이트!

Q

array([[3.97443354e-03, 6.00913792e-03, 3.58364769e-03, 3.82618673e-03],
       [4.69615167e-03, 0.00000000e+00, 7.86154285e-03, 2.75337187e-03],
       [4.14476568e-03, 1.90846936e-02, 2.10744352e-03, 1.23311967e-02],
       [4.73115881e-03, 0.00000000e+00, 6.77601001e-04, 0.00000000e+00],
       [5.41804613e-03, 1.90232900e-02, 0.00000000e+00, 2.02776331e-03],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 8.62754097e-02, 0.00000000e+00, 6.25230675e-03],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.03396279e-02, 0.00000000e+00, 6.00199345e-02, 6.60239186e-03],
       [2.61101550e-02, 1.33484782e-01, 1.00671397e-01, 0.00000000e+00],
       [4.01889794e-02, 3.78146617e-01, 0.00000000e+00, 2.77332194e-02],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 1.24110730e-01, 3.50621207e

# 자! Action-Value 바탕으로 움직인다면?

바꿔 말해, Policy를 제작한다면?

$$\pi'(s) = argmax_{a}(Q(s,a) )$$

In [26]:
print(Q[4])
print(Q[4].argmax())

[0.00541805 0.01902329 0.         0.00202776]
1


In [28]:
s0 = env.reset()
env.render()
for i in range(1000):
    a0 = Q[s0].argmax()
    s1, r1, done, _ = env.step(a0)
    env.render()
    if done == True :
        env.close()
        break
    s0 = s1


SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
